## Author: Wonho Lim 
#### E-mail: wonholim02@gmail.com
#### Environment: Python 3.10.4 (ipykernel)
#### Last Update: 2022-05-18
#### Description: Web crawling code for EURONEXT.com, especially for EURONEXT Amsterdam. Can be reused for other country by simple editing.

### Utilized BS4 and Selenium Web for Crawling - Visualized by Pandas

In [1]:
import bs4
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import csv 
import pandas as pd 
from platform import python_version

### Environment
#### - Chromdriver must be located in local PC that is running. This must be modified to be used.

In [2]:
path = r'C:\Users\user\Desktop\자료\chromedriver'

In [3]:
print(python_version())

3.9.12


In [4]:
print(pd.__version__)

1.4.2


### Basic setting for csv file that will store 재무정보 data - Order of rows are based on 재무정보 데이터소스

In [5]:
cover = open('Netherlands_2022_EURONEXT_2022-05-23_Chris.csv','w', newline='', encoding='CP949')
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드', '현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

### Crawled Website
EURONEXT MAIN: https://live.euronext.com/en /
EURONEXT AMSTERDAM: https://live.euronext.com/en/markets/amsterdam/equities/list

## This is the main code that crawls data rom EURONEXT. 
#### - Time sleep was necessary because of comparably long loading time.
#### - Uncollectable or unavailable data was displayed as "(NA) DataName."
#### - (계산예정) Data Will be calculated and inserted manually later.
#### - Sample Visualization is located in the bottom.

In [15]:
failList = []
#EURONEXT 
driver = webdriver.Chrome(path)
driver.get('https://live.euronext.com/en/markets/amsterdam/equities/list')
time.sleep(2)
driver.find_element(By.XPATH, value='/html/body/div[3]/div/div/div[3]/button[1]').click()
base_url = "https://live.euronext.com"

#Get each company's link, market symbol, and market code on table - P1
time.sleep(4)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
time.sleep(4)
all_atag_maintable = maintable.find_all('a')
all_market_maintable = maintable.find_all(attrs={'td',"stocks-market"})
all_symbol_maintable = maintable.find_all(attrs={'td',"stocks-symbol"})

#We have to process the collected data again because of their special structure
newList3 =[]
for a in all_market_maintable :
    newList3.append(a.text)
newList4 =[]
for b in all_symbol_maintable :
    newList4.append(b.text)

#Get each company's link, market symbol, and market code on table - P2
driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/span/a[2]').click()
time.sleep(4)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable2 = soup.find('table', id='stocks-data-table')
time.sleep(4)
all_atag_maintable2 = maintable2.find_all('a')
all_market_maintable2 = maintable2.find_all(attrs={'td',"nowrap pointer"})
all_symbol_maintable2 = maintable2.find_all(attrs={'td',"stocks-symbol"})

#We have to process the collected data again because of their special structure
newList1 =[]
for c in all_market_maintable2 :
    newList1.append(c.text)

newList2 =[]
for d in all_symbol_maintable2 :
    newList2.append(d.text)
    
#Merge two lists (from page 1 and 2)
all_atag_maintable = all_atag_maintable + all_atag_maintable2
all_symbol_maintable = newList3 + newList1
all_market_maintable = newList4 + newList2

#Final Processing to remove unnecessary data
all_symbol_maintable.pop(0)
all_market_maintable.pop(0)
company_links = []
failList = []
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)
    

# To check if all companies are selected
#print(len(company_links))

# Main Data Collection code
for sub in company_links:
    driver.get(base_url + sub)
    info = []
    time.sleep(4)
    
    #국가코드
    info.append("NLD")
    info.append("Nederland")
    info.append("Netherlands")
    info.append("UTC+01:00")
    info.append("Europe")
    info.append("912200000000 USD")
    info.append("17440000")
    info.append("서유럽")
    
    #기업
    name = driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text
    info.append("NLD XAMS" + name)
    info.append(name)
    info.append(name)
    
    #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
    try :
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
        time.sleep(4)
        address = driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[1]/div[2]').text
        try:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: " + address)
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: " + address)
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K Euros. You can find more information on company's website. Company Address: " + address)
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K Euros. You can find more information on company's website. Company Address: " + address)
        except:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.")
        info.append("")
        info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[3]/div/div/div/div[2]/div[2]/a').text) 
        except:
            try: 
                info.append(driver.find_elemen(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text)
            except:
                info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[3]/a').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[3]/div/a').text)
        except:
            info.append(str(base_url + sub))
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
        except:
            ("Nederland")
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
        except:
            info.append("Netherlands")
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
        except:
            info.append("Nederland, Europe")
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
        except:
            info.append("Netherlands, Europe")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("Active") 
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[1]').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[1]').text)
        except:
            info.append("")
            info.append("")
        info.append("CONTACT")
        info.append("CONTACT")
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[1]/div/div/div/div/div/div[1]/div[1]/h1/strong').text)
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text) 
        except:
            try: info.append(driver.find_elemen(By.XPATH, value ='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/address/div[2]/div[1]/a').text)
            except:
                info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text)
        except:
            info.append("")  
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text) 
        except:
            try: 
                info.append(driver.find_elemen(By.XPATH, value ='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[1]/a').text)
            except:
                info.append("")
    except: 
        ## 소개내용
        info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).")
        info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).")
        info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.")
        info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(str(base_url + sub))
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("Nederland")
        info.append("Netherlands")
        info.append("Nederland, Europe")
        info.append("Netherlands, Europe")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("Active") 
        info.append("")
        info.append("")
        info.append("CONTACT")
        info.append("CONTACT")
        info.append("")
        info.append("")
        info.append("")
        info.append(str(base_url + sub))
        info.append("")
        
    #재무정보
    try:
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
        time.sleep(4)
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/thead/tr/th[2]').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[13]/td').text)
            except:
                info.append("")  
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "천 유로")
        except:
            info.append("")  
        info.append("")
        info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[3]/td[2]').text + "천 유로")
        except:
            info.append("")  
        info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[4]/td[2]').text + "천 유로")
        except:
            info.append("")  
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[7]/td[2]').text + "천 유로")      
        except:
            info.append("") 
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
    except : 
        info.append("")  
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")  
        info.append("")
        info.append("")
        info.append("")  
        info.append("")
        info.append("")  
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("") 
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

    #재무정보중 계산예정            
    info.append("")
    info.append("")
    info.append("")
    info.append("")
    info.append("")
    
    #산업군
    try:
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[2]').click()
        time.sleep(4)
        try : 
            info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
            info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
        except :
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
            except:
                info.append("")
                info.append("")
        try :
            info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
            info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
        except :
            try: 
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
            except:
                info.append("")
                info.append("")
    except:
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        
    #언어
    info.append("NL")
    info.append("Nederlands")
    info.append("Dutch")
    
    #주식시장정보
    info.append(str(all_symbol_maintable.pop(0)))
    info.append("EURONEXT AMSTERDAM")
    info.append("EURONEXT AMSTERDAM")
    info.append(str(all_market_maintable.pop(0)))
    info.append("")
    driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[1]').click()
    time.sleep(4)
    try:
        info.append(driver.find_element(By.CSS_SELECTOR, value='#header-instrument-price').text + " EUR")
    except:
        info.append("")
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[2]').text  + " EUR")
    except:
        info.append("")
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[3]').text  + " EUR")
    except:
        info.append("")
    info.append("")
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[5]').text  + " EUR")
    except:
        info.append("")
    info.append("")
    info.append("")
    info.append("")
    
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[3]/td[2]').text)
    except:
        info.append("")
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[12]/td[2]').text)
    except:
        info.append("")

    #지점
    info.append("")
    info.append("")
    info.append("Netherlands")
    info.append("Netherlands, Europe")
    info.append("")
    info.append("")
    
    #이벤트
    try:
        element = driver.find_element(By.CSS_SELECTOR, value='#content-left > section > div.container-full > div.card.mb-4.text-left > div.card-body > div > div > div > table > tbody > tr:nth-child(1) > td:nth-child(2) > a')
        href = element.get_attribute('href')
        info.append(href)
    except:
        info.append("")
        
    #화폐
    info.append("EUR")
    info.append("유로")
    
    #관리
    info.append("Chris")
    info.append("EURONEXT")
    info.append("2022-05-26")
    
    #Keep data in csv
    print(info)
    try: 
        writing.writerow(info);
    except:
        failList.append(info);

C:\Users\user\AppData\Local\Temp\ipykernel_17792\3321949143.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


['NLD', 'Nederland', 'Netherlands', 'UTC+01:00', 'Europe', '912200000000 USD', '17440000', '서유럽', 'NLD XAMSAALBERTS N.V.', 'AALBERTS N.V.', 'AALBERTS N.V.', 'AALBERTS N.V.(English: AALBERTS N.V.) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: Stadsplateau 18', 'AALBERTS N.V.(English: AALBERTS N.V.) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: Stadsplateau 18', "AALBERTS N.V.(English: AALBERTS N.V.) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were 2.979.100K Euros. You can find more information on company's website. Company Address: Stadsplateau 18", "AALBERTS N.V.(English: AALBERTS N.V.) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were 2.979.100K Euros. You can find more information on company's website. Company Address: Stadsplateau 18", '', '', '', '', '', 'jeannette.zuidema@aalberts.com', 'https://aalberts.com/investor', '', '', ''

['NLD', 'Nederland', 'Netherlands', 'UTC+01:00', 'Europe', '912200000000 USD', '17440000', '서유럽', 'NLD XAMSADYEN', 'ADYEN', 'ADYEN', 'ADYEN(English: ADYEN) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).', 'ADYEN(English: ADYEN) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).', "ADYEN(English: ADYEN) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.", "ADYEN(English: ADYEN) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.", '', '', '', '', '', '', 'https://live.euronext.com/en/product/equities/NL0012969182-XAMS/adyen/adyen/quotes', '', '', '', '', '', 'Nederland', 'Netherlands', 'Nederland, Europe', 'Netherlands, Europe', '', '', '', '', 'Active', '', '', 'CONTACT', 'CONTACT', '', '', '', 'https://live.euronext.com/en/product/equities/NL0012969182-XAMS/adyen/adyen/quotes', '', '2021', '', '', '',

['NLD', 'Nederland', 'Netherlands', 'UTC+01:00', 'Europe', '912200000000 USD', '17440000', '서유럽', 'NLD XAMSAJAX', 'AJAX', 'AJAX', 'AJAX(English: AJAX) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: Arena 29 Boulevard', 'AJAX(English: AJAX) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: Arena 29 Boulevard', "AJAX(English: AJAX) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were 125.159K Euros. You can find more information on company's website. Company Address: Arena 29 Boulevard", "AJAX(English: AJAX) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were 125.159K Euros. You can find more information on company's website. Company Address: Arena 29 Boulevard", '', '', '', '', '+31 (0) 20 311 14 80', '', 'http://www.ajax.nl', '', '', '', '', '', 'Arena 29 Boulevard', 'Arena 29 Boulevard', 'Po Box 12522', 'Po Box 12522', '', '', '', '', 'Active', 

['NLD', 'Nederland', 'Netherlands', 'UTC+01:00', 'Europe', '912200000000 USD', '17440000', '서유럽', 'NLD XAMSALUMEXX N.V.', 'ALUMEXX N.V.', 'ALUMEXX N.V.', 'ALUMEXX N.V.(English: ALUMEXX N.V.) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).', 'ALUMEXX N.V.(English: ALUMEXX N.V.) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).', "ALUMEXX N.V.(English: ALUMEXX N.V.) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.", "ALUMEXX N.V.(English: ALUMEXX N.V.) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.", '', '', '', '', '', '', 'https://live.euronext.com/en/product/equities/NL0012194724-XAMS/alumexx-nv/alx/quotes', '', '', '', '', '', 'Nederland', 'Netherlands', 'Nederland, Europe', 'Netherlands, Europe', '', '', '', '', 'Active', '', '', 'CONTACT', 'CONTACT', '', '', '', 'https://live.euronext.com/en

...

['NLD', 'Nederland', 'Netherlands', 'UTC+01:00', 'Europe', '912200000000 USD', '17440000', '서유럽', 'NLD XAMSSPEAR INVESTMENTS I WARRANTS', 'SPEAR INVESTMENTS I WARRANTS', 'SPEAR INVESTMENTS I WARRANTS', 'SPEAR INVESTMENTS I WARRANTS(English: SPEAR INVESTMENTS I WARRANTS) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).', 'SPEAR INVESTMENTS I WARRANTS(English: SPEAR INVESTMENTS I WARRANTS) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).', "SPEAR INVESTMENTS I WARRANTS(English: SPEAR INVESTMENTS I WARRANTS) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.", "SPEAR INVESTMENTS I WARRANTS(English: SPEAR INVESTMENTS I WARRANTS) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.", '', '', '', '', '', '', 'https://live.euronext.com/en/product/equities/NL0015000NK1-XAMS/spr1w-warrants/spr1w/quotes', '', '',

['NLD', 'Nederland', 'Netherlands', 'UTC+01:00', 'Europe', '912200000000 USD', '17440000', '서유럽', 'NLD XAMSTOMTOM', 'TOMTOM', 'TOMTOM', 'TOMTOM(English: TOMTOM) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).', 'TOMTOM(English: TOMTOM) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).', "TOMTOM(English: TOMTOM) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.", "TOMTOM(English: TOMTOM) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.", '', '', '', '', '', '', 'https://live.euronext.com/en/product/equities/NL0013332471-XAMS/tomtom/tom2/quotes', '', '', '', '', '', 'Nederland', 'Netherlands', 'Nederland, Europe', 'Netherlands, Europe', '', '', '', '', 'Active', '', '', 'CONTACT', 'CONTACT', '', '', '', 'https://live.euronext.com/en/product/equities/NL0013332471-XAMS/tomtom/tom2/quotes', '', '2021', 

['NLD', 'Nederland', 'Netherlands', 'UTC+01:00', 'Europe', '912200000000 USD', '17440000', '서유럽', 'NLD XAMSVALUE8', 'VALUE8', 'VALUE8', 'VALUE8(English: VALUE8) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: Brediusweg 33', 'VALUE8(English: VALUE8) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: Brediusweg 33', "VALUE8(English: VALUE8) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were 20.903K Euros. You can find more information on company's website. Company Address: Brediusweg 33", "VALUE8(English: VALUE8) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were 20.903K Euros. You can find more information on company's website. Company Address: Brediusweg 33", '', '', '', '', '', '', 'http://www.value8.com', '', '', '', '', '', 'Brediusweg 33', 'Brediusweg 33', 'PO Box 26', 'PO Box 26', '', '', '', '', 'Active', '', '', 'CONTACT', 'CONTACT', 'VA

['NLD', 'Nederland', 'Netherlands', 'UTC+01:00', 'Europe', '912200000000 USD', '17440000', '서유럽', 'NLD XAMSVASTNED RETAIL N.V.', 'VASTNED RETAIL N.V.', 'VASTNED RETAIL N.V.', 'VASTNED RETAIL N.V.(English: VASTNED RETAIL N.V.) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).', 'VASTNED RETAIL N.V.(English: VASTNED RETAIL N.V.) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).', "VASTNED RETAIL N.V.(English: VASTNED RETAIL N.V.) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.", "VASTNED RETAIL N.V.(English: VASTNED RETAIL N.V.) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.", '', '', '', '', '', '', 'https://live.euronext.com/en/product/equities/NL0000288918-XAMS/vastned/vastn/quotes', '', '', '', '', '', 'Nederland', 'Netherlands', 'Nederland, Europe', 'Netherlands, Europe', '', '', '', '', 'Acti

['NLD', 'Nederland', 'Netherlands', 'UTC+01:00', 'Europe', '912200000000 USD', '17440000', '서유럽', 'NLD XAMSWAREHOUSES DE PAUW', 'WAREHOUSES DE PAUW', 'WAREHOUSES DE PAUW', 'WAREHOUSES DE PAUW(English: WAREHOUSES DE PAUW) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).', 'WAREHOUSES DE PAUW(English: WAREHOUSES DE PAUW) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands).', "WAREHOUSES DE PAUW(English: WAREHOUSES DE PAUW) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.", "WAREHOUSES DE PAUW(English: WAREHOUSES DE PAUW) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). You can find more information on company's website.", '', '', '', '', '', '', 'https://live.euronext.com/en/product/equities/BE0974349814-XBRU/wdp/wdp/quotes', '', '', '', '', '', 'Nederland', 'Netherlands', 'Nederland, Europe', 'Netherlands, Europe', '', '', '', '', 'Active', '', '', 'CON

In [11]:
print(len(failList))
print(failList)

5
[['NLD', 'Nederland', 'Netherlands', 'UTC+01:00', 'Europe', '912200000000 USD', '17440000', '서유럽', 'NLD XAMSAIR FRANCE -KLM', 'AIR FRANCE -KLM', 'AIR FRANCE -KLM', 'AIR FRANCE -KLM(English: AIR FRANCE -KLM) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: 2 Rue de Robert-Esnault-Pelterie', 'AIR FRANCE -KLM(English: AIR FRANCE -KLM) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: 2 Rue de Robert-Esnault-Pelterie', "AIR FRANCE -KLM(English: AIR FRANCE -KLM) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were 14.315.000K Euros. You can find more information on company's website. Company Address: 2 Rue de Robert-Esnault-Pelterie", "AIR FRANCE -KLM(English: AIR FRANCE -KLM) is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were 14.315.000K Euros. You can find more information on company's website. Company Address: 2 Rue de Robert-Esnault-Pelterie", ''

## Data Visualization

##### By rows

In [12]:
with open("DutchData.csv", "r") as table:
    for row in table:
        print(row)

헤브론스타국가코드,현지언어국가명,영문국가명,시간,대륙,GDP,인구,지역,기업식별코드,현지언어기업명,영문기업명,현지언어한줄소개내용,영문한줄소개내용,현지언어기업소개내용,영문기업소개내용,설립일자,법인등록번호,사업자등록번호,기업대표전화번호,대표팩스번호,대표이메일,기업홈페이지URL,페이스북URL,인스타그램URL,유튜브URL,링크드인URL,트위터핸들,현지언어기업주소,영문기업주소,현지언어기업상세주소,영문기업상세주소,기업우편번호,기업종업원,외감법인구분,기업연수,기업상태,현지언어담당자명,영문담당자명,현지언어직위명,영문직위명,담당자부서명,담당자전화번호,담당자팩스번호,담당자이메일,담당자이동전화번호,회계연도,유동자산금액,비유동자산금액,자산총계금액,유동부채금액,비유동부채금액,부채총계금액,자본총계금액,부채자본총계금액,매출액,매출원가금액,판매비관리비금액,영업이익손실금액,금융수익금액,금융비용금액,기타영업외수익금액,기타영업외비용금액,법인세차감전순이익,법인세비용,당기순이익,영업활동현금흐름금액,투자활동현금흐름금액,재무활동현금흐름금액,기초현금자산금액,기말현금자산금액,부채비율,영업이익율,매출액증가율,영업이익증가율,당기순이익 증가율,기업 CAGR,현지언어산업군명,영문산업군명,현지언어주요제품명내용,영문주요제품명내용,국가언어코드,현지언어언어명,영문언어명,주식시장코드,현지언어주식시장명,영문주식시장명,상장코드,상장일자,주가(일),주가(1주),주가(1개월),주가(6개월),주가(1년),주가(3년),주가(5년),주가(10년),거래량,시가총액,지점코드,지점명,주소,주소상세,우편번호,사업자등록번호,이벤트,통화구분코드,화폐단위명,담당자,소스,날짜

NLD,Nederland,Netherlands,UTC+01:00,Europe,912200000000 USD,17440000,서유럽,NLD XAMSAALBERTS N.V.,AALBERTS N.V.,AALBERTS N.V.,AALBERTS N.V.(English: AALBERTS N.V.) is a public company that is listed on EURONEXT A

##### Headers

In [13]:
file = open("DutchData.csv")
csvreader = csv.reader(file)
header = next(csvreader)
print(header)

['헤브론스타국가코드', '현지언어국가명', '영문국가명', '시간', '대륙', 'GDP', '인구', '지역', '기업식별코드', '현지언어기업명', '영문기업명', '현지언어한줄소개내용', '영문한줄소개내용', '현지언어기업소개내용', '영문기업소개내용', '설립일자', '법인등록번호', '사업자등록번호', '기업대표전화번호', '대표팩스번호', '대표이메일', '기업홈페이지URL', '페이스북URL', '인스타그램URL', '유튜브URL', '링크드인URL', '트위터핸들', '현지언어기업주소', '영문기업주소', '현지언어기업상세주소', '영문기업상세주소', '기업우편번호', '기업종업원', '외감법인구분', '기업연수', '기업상태', '현지언어담당자명', '영문담당자명', '현지언어직위명', '영문직위명', '담당자부서명', '담당자전화번호', '담당자팩스번호', '담당자이메일', '담당자이동전화번호', '회계연도', '유동자산금액', '비유동자산금액', '자산총계금액', '유동부채금액', '비유동부채금액', '부채총계금액', '자본총계금액', '부채자본총계금액', '매출액', '매출원가금액', '판매비관리비금액', '영업이익손실금액', '금융수익금액', '금융비용금액', '기타영업외수익금액', '기타영업외비용금액', '법인세차감전순이익', '법인세비용', '당기순이익', '영업활동현금흐름금액', '투자활동현금흐름금액', '재무활동현금흐름금액', '기초현금자산금액', '기말현금자산금액', '부채비율', '영업이익율', '매출액증가율', '영업이익증가율', '당기순이익 증가율', '기업 CAGR', '현지언어산업군명', '영문산업군명', '현지언어주요제품명내용', '영문주요제품명내용', '국가언어코드', '현지언어언어명', '영문언어명', '주식시장코드', '현지언어주식시장명', '영문주식시장명', '상장코드', '상장일자', '주가(일)', '주가(1주)', '주가(1개월)', '주가(6개월)', '주가(1년)', '주가(3년)', '주가(5년)'